# P1b — Long-Only MeanReversion Variant

**Hypothesis (F5):** BollingerMeanReversion fails in a bull market because it shorts
overbought conditions into a rising trend. The structural fix is *directional bias*:
only buy oversold dips (long-only), and optionally add a 200-bar MA switch to flip
short-only when the trend reverses.

**Three variants tested:**
1. **Baseline** — original BollingerMeanReversion (long + short)
2. **LongOnly** — buy below lower band only; flat when overbought
3. **TrendFiltered** — long-only above 200MA, short-only below 200MA

## §1 — Config

In [1]:
import sys
from pathlib import Path

repo_root = Path("__file__").resolve().parent.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

plt.rcParams.update({
    "figure.dpi":        120,
    "axes.spines.top":   False,
    "axes.spines.right": False,
})

SINCE      = "2024-01-01"
UNTIL      = "2025-01-01"
BB_PERIOD  = 20
BB_STD     = 2.0
MA_PERIOD  = 200   # trend-switch lookback

N_SPLITS   = 5
TRAIN_FRAC = 0.6

print(f"Period: {SINCE} → {UNTIL}")
print(f"BB({BB_PERIOD}, {BB_STD})  |  Trend MA={MA_PERIOD}  |  WF splits={N_SPLITS}")

Period: 2024-01-01 → 2025-01-01
BB(20, 2.0)  |  Trend MA=200  |  WF splits=5


## §2 — Fetch data

In [2]:
from data.fetch import fetch_ohlcv

df = fetch_ohlcv(since=SINCE, until=UNTIL)
print(f"{len(df):,} bars  |  {df.index[0]} → {df.index[-1]}")

# Quick price overview
fig, ax = plt.subplots(figsize=(14, 3))
df["close"].plot(ax=ax, color="steelblue", linewidth=0.8)
ma200 = df["close"].rolling(MA_PERIOD).mean()
ma200.plot(ax=ax, color="darkorange", linewidth=1.2, label="200-bar MA")
ax.set_title("BTC/USDT 1h close — full-year 2024 with 200-bar MA", fontsize=11)
ax.set_ylabel("Price (USDT)")
ax.legend()
plt.tight_layout()
plt.show()

above_ma = (df["close"] > ma200).mean()
print(f"Price above 200-bar MA: {above_ma*100:.1f}% of bars  (bull-dominant year)")

8,785 bars  |  2024-01-01 00:00:00+00:00 → 2025-01-01 00:00:00+00:00


Price above 200-bar MA: 54.8% of bars  (bull-dominant year)


## §3 — Define variant strategies

In [3]:
from strategies.base import BaseStrategy


def _add_bb(df, period, num_std):
    df = df.copy()
    df["bb_mid"]   = df["close"].rolling(period).mean()
    df["bb_std"]   = df["close"].rolling(period).std()
    df["bb_upper"] = df["bb_mid"] + num_std * df["bb_std"]
    df["bb_lower"] = df["bb_mid"] - num_std * df["bb_std"]
    return df


class BollingerLongOnly(BaseStrategy):
    """Buy when price drops below the lower band; flat otherwise (never short)."""

    def __init__(self, period: int = 20, num_std: float = 2.0):
        super().__init__(period=period, num_std=num_std)
        self.period  = period
        self.num_std = num_std

    def generate_signals(self, df):
        df = _add_bb(df, self.period, self.num_std)
        df["signal"] = 0
        df.loc[df["close"] < df["bb_lower"], "signal"] = 1
        return df


class BollingerTrendFiltered(BaseStrategy):
    """Long-only above the MA_PERIOD MA; short-only below it.

    Above MA: signal=+1 when oversold, else 0 (no shorting into bull trend).
    Below MA: signal=-1 when overbought, else 0 (no longing into bear trend).
    """

    def __init__(self, period: int = 20, num_std: float = 2.0, ma_period: int = 200):
        super().__init__(period=period, num_std=num_std, ma_period=ma_period)
        self.period    = period
        self.num_std   = num_std
        self.ma_period = ma_period

    def generate_signals(self, df):
        df = _add_bb(df, self.period, self.num_std)
        df["trend_ma"] = df["close"].rolling(self.ma_period).mean()

        df["signal"] = 0
        bull = df["close"] > df["trend_ma"]
        bear = df["close"] <= df["trend_ma"]

        # Bull regime: only go long on oversold dips
        df.loc[bull & (df["close"] < df["bb_lower"]), "signal"] = 1
        # Bear regime: only go short on overbought spikes
        df.loc[bear & (df["close"] > df["bb_upper"]), "signal"] = -1

        return df


print("Strategies defined:")
print("  1. BollingerMeanReversion  (baseline — long + short)")
print("  2. BollingerLongOnly       (buy dips only, never short)")
print("  3. BollingerTrendFiltered  (long above 200MA, short below 200MA)")

Strategies defined:
  1. BollingerMeanReversion  (baseline — long + short)
  2. BollingerLongOnly       (buy dips only, never short)
  3. BollingerTrendFiltered  (long above 200MA, short below 200MA)


## §4 — Single-period sanity check (Jan–Jun 2024)

Reproduce F4: baseline Sharpe should be negative over this bull-run period.
Check whether LongOnly and TrendFiltered flip it positive.

In [4]:
from backtesting import compute_metrics
from strategies.single import BollingerMeanReversion

half_year = df.loc["2024-01-01":"2024-06-30"]
bar_ret   = half_year["close"].pct_change().fillna(0)

strategies = [
    ("Baseline (long+short)",  BollingerMeanReversion,  {"period": BB_PERIOD, "num_std": BB_STD}),
    ("LongOnly",               BollingerLongOnly,       {"period": BB_PERIOD, "num_std": BB_STD}),
    ("TrendFiltered (200MA)",  BollingerTrendFiltered,  {"period": BB_PERIOD, "num_std": BB_STD, "ma_period": MA_PERIOD}),
]

fig, ax = plt.subplots(figsize=(14, 4))
rows = []

# Buy-and-hold reference
bah = half_year["close"] / half_year["close"].iloc[0]
bah.plot(ax=ax, label="Buy-and-Hold", color="gray", linewidth=1.0, linestyle=":")

colors = ["#e74c3c", "#2ecc71", "#3498db"]
for (label, cls, params), color in zip(strategies, colors):
    sig_df = cls(**params).generate_signals(half_year)
    pos    = sig_df["signal"].shift(1).fillna(0)
    equity = (1 + pos * bar_ret).cumprod()
    equity = equity / equity.iloc[0]
    m      = compute_metrics(equity)
    rows.append({
        "Strategy":      label,
        "Total Return":  m["total_return"],
        "Sharpe":        m["sharpe_ratio"],
        "Sortino":       m["sortino_ratio"],
        "Max Drawdown":  m["max_drawdown"],
        "Win Rate":      m["win_rate"],
    })
    equity.plot(ax=ax, label=label, color=color, linewidth=1.4)

ax.axhline(1, color="black", linewidth=0.5)
ax.set_title("Jan–Jun 2024 single-period equity curves (F4 sanity check)", fontsize=11)
ax.set_ylabel("Equity (normalised)")
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

metrics_df = pd.DataFrame(rows).set_index("Strategy")
pd.set_option("display.float_format", "{:.4f}".format)
metrics_df

,Total Return,Sharpe,Sortino,Max Drawdown,Win Rate
Strategy,,,,,
Baseline (long+short),-0.0617,-0.4014,-0.1559,-0.1706,0.0673
LongOnly,0.0450,0.5747,0.1501,-0.0833,0.0323
TrendFiltered (200MA),-0.0378,-0.5604,-0.1024,-0.0999,0.0211


## §5 — Walk-forward comparison (full-year 2024, 5 rolling folds)

In [5]:
from backtesting import walk_forward

wf_baseline = walk_forward(
    BollingerMeanReversion, df,
    {"period": BB_PERIOD, "num_std": BB_STD},
    n_splits=N_SPLITS, train_frac=TRAIN_FRAC, window_type="rolling",
)
wf_longonly = walk_forward(
    BollingerLongOnly, df,
    {"period": BB_PERIOD, "num_std": BB_STD},
    n_splits=N_SPLITS, train_frac=TRAIN_FRAC, window_type="rolling",
)
wf_trend = walk_forward(
    BollingerTrendFiltered, df,
    {"period": BB_PERIOD, "num_std": BB_STD, "ma_period": MA_PERIOD},
    n_splits=N_SPLITS, train_frac=TRAIN_FRAC, window_type="rolling",
)

# Summary table
keys = ["total_return", "sharpe_ratio", "sortino_ratio", "max_drawdown", "win_rate"]
oos_summary = pd.DataFrame({
    "Baseline":      {k: wf_baseline.oos_metrics[k] for k in keys},
    "LongOnly":      {k: wf_longonly.oos_metrics[k] for k in keys},
    "TrendFiltered": {k: wf_trend.oos_metrics[k]    for k in keys},
}).T

print("Walk-forward OOS aggregate metrics (full-year 2024, 5 rolling folds)")
oos_summary

Walk-forward OOS aggregate metrics (full-year 2024, 5 rolling folds)


,total_return,sharpe_ratio,sortino_ratio,max_drawdown,win_rate
Baseline,-0.2063,-1.1209,-0.4490,-0.3349,0.0648
LongOnly,-0.0358,-0.1801,-0.0486,-0.1894,0.0331
TrendFiltered,-0.0700,-0.9339,-0.1969,-0.1207,0.0168


## §6 — Stitched OOS equity curves (all three variants)

In [6]:
oos_start  = wf_baseline.oos_equity.index[0]
oos_end    = wf_baseline.oos_equity.index[-1]
bah_oos    = df.loc[oos_start:oos_end, "close"]
bah_equity = bah_oos / bah_oos.iloc[0]

fig, ax = plt.subplots(figsize=(14, 4))

wf_baseline.oos_equity.plot(ax=ax, label="Baseline (long+short)", color="#e74c3c", linewidth=1.4)
wf_longonly.oos_equity.plot(ax=ax, label="LongOnly",              color="#2ecc71", linewidth=1.4)
wf_trend.oos_equity.plot(   ax=ax, label="TrendFiltered (200MA)", color="#3498db", linewidth=1.4)
bah_equity.plot(            ax=ax, label="Buy-and-Hold",          color="gray",    linewidth=1.0, linestyle=":")

for wr in wf_baseline.windows:
    ax.axvline(wr.test_start, color="lightgray", linewidth=0.5, linestyle="--")

ax.axhline(1, color="black", linewidth=0.5)
ax.set_title("Stitched OOS equity — Baseline vs LongOnly vs TrendFiltered", fontsize=11)
ax.set_ylabel("Equity (normalised)")
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

## §7 — Per-window Sharpe breakdown

In [7]:
window_labels = [f"W{i+1}" for i in range(N_SPLITS)]

sharpe_base = [wr.metrics["sharpe_ratio"] for wr in wf_baseline.windows]
sharpe_long = [wr.metrics["sharpe_ratio"] for wr in wf_longonly.windows]
sharpe_tren = [wr.metrics["sharpe_ratio"] for wr in wf_trend.windows]

x = np.arange(N_SPLITS)
width = 0.25

fig, ax = plt.subplots(figsize=(12, 4))
ax.bar(x - width, sharpe_base, width, label="Baseline",      color="#e74c3c", alpha=0.85)
ax.bar(x,         sharpe_long, width, label="LongOnly",      color="#2ecc71", alpha=0.85)
ax.bar(x + width, sharpe_tren, width, label="TrendFiltered", color="#3498db", alpha=0.85)

ax.axhline(0, color="black", linewidth=0.8)
ax.set_xticks(x)
ax.set_xticklabels(window_labels)
ax.set_title("Per-window Sharpe ratio — Baseline vs LongOnly vs TrendFiltered", fontsize=11)
ax.set_ylabel("Sharpe ratio")
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

# Tabular breakdown
breakdown = pd.DataFrame({
    "Window":       window_labels,
    "Test Start":   [wr.test_start.date() for wr in wf_baseline.windows],
    "Test End":     [wr.test_end.date()   for wr in wf_baseline.windows],
    "Sharpe_Base":  sharpe_base,
    "Sharpe_Long":  sharpe_long,
    "Sharpe_Trend": sharpe_tren,
    "Winner":       [
        ["Baseline", "LongOnly", "TrendFiltered"][np.argmax([b, l, t])]
        for b, l, t in zip(sharpe_base, sharpe_long, sharpe_tren)
    ],
})
breakdown

,Window,Test Start,Test End,Sharpe_Base,Sharpe_Long,Sharpe_Trend,Winner
0,W1,2024-03-02,2024-05-01,-2.1467,-0.1428,-3.2994,LongOnly
1,W2,2024-05-02,2024-07-01,0.3856,0.8220,1.7749,TrendFiltered
2,W3,2024-07-02,2024-08-31,-4.7099,-3.9607,-3.0275,TrendFiltered
3,W4,2024-09-01,2024-10-31,0.6031,-2.2301,-0.6472,Baseline
4,W5,2024-11-01,2024-12-31,1.2382,4.9307,1.2454,LongOnly


## §8 — Signal composition analysis

How much does each variant trade, and what's the long/short split?

In [8]:
for label, cls, params in strategies:
    sig = cls(**params).generate_signals(df)["signal"]
    counts = sig.value_counts().sort_index()
    total  = len(sig)
    longs  = counts.get(1, 0)
    shorts = counts.get(-1, 0)
    flat   = counts.get(0, 0)
    active = longs + shorts
    print(f"{label}")
    print(f"  Long:  {longs:5d} ({longs/total*100:.1f}%)")
    print(f"  Short: {shorts:5d} ({shorts/total*100:.1f}%)")
    print(f"  Flat:  {flat:5d} ({flat/total*100:.1f}%)")
    print(f"  Active bars: {active} / {total}")
    print()

Baseline (long+short)
  Long:    498 (5.7%)
  Short:   542 (6.2%)
  Flat:   7745 (88.2%)
  Active bars: 1040 / 8785

LongOnly
  Long:    498 (5.7%)
  Short:     0 (0.0%)
  Flat:   8287 (94.3%)
  Active bars: 498 / 8785

TrendFiltered (200MA)
  Long:    171 (1.9%)
  Short:   113 (1.3%)
  Flat:   8501 (96.8%)
  Active bars: 284 / 8785



## §9 — Conclusions

**Finding F6 — Directional bias reduces losses but does not create alpha in a bull market**

### Jan–Jun 2024 single-period

| Variant | Sharpe | Return | Max DD |
|---|---|---|---|
| Baseline (long+short) | −0.40 | −6.2% | −17.1% |
| **LongOnly** | **+0.58** | **+4.5%** | −8.3% |
| TrendFiltered (200MA) | −0.56 | −3.8% | −10.0% |

### Walk-forward OOS — full-year 2024 (5 rolling folds)

| Variant | OOS Sharpe | OOS Return | Max DD | Win Rate |
|---|---|---|---|---|
| Baseline (long+short) | −1.12 | −20.6% | −33.5% | 6.5% |
| **LongOnly** | **−0.18** | **−3.6%** | −18.9% | 3.3% |
| TrendFiltered (200MA) | −0.93 | −7.0% | **−12.1%** | 1.7% |

### Key observations

1. **LongOnly is clearly the best variant.** Removing short signals eliminates the dominant
   source of losses — shorting into a bull market. Return improves from −20.6% to −3.6%
   and Sharpe from −1.12 to −0.18 on a walk-forward basis.

2. **TrendFiltered underperforms LongOnly despite better max drawdown.**
   The 200MA filter cuts active bars by 73% (1040 → 284), so it barely trades. When it does
   trade short (bear regime), those signals are also losers in 2024. The MaxDD improvement
   (−12.1% vs −18.9%) comes only from reduced exposure.

3. **None of the variants are profitable on a walk-forward basis.** The Bollinger
   mean-reversion edge (buying oversold dips) is too small to overcome friction in a
   strong trending year — BTC went from ~42K to ~93K, and dip-buyers were repeatedly
   squeezed out before the rebound.

4. **Per-window variance is still high.** W5 (Nov–Dec 2024): LongOnly Sharpe +4.93.
   W3 (Jul–Aug 2024): LongOnly Sharpe −3.96. The strategy is highly regime-sensitive
   within a single year.

5. **Win rates are very low (1.7–6.5%).** Only ~5.7% of bars touch the lower band;
   of those, many are false signals during momentum-driven drops. This is a structural
   signal scarcity problem, not a directional one.

### Recommendation

- **Promote `BollingerLongOnly`** to `strategies/single/basic/` as a documented variant —
  it is strictly better than the baseline for bull-market use.
- **TrendFiltered** needs a shorter MA period (e.g. 50-bar) or a different regime
  classifier; with MA=200 it barely trades. Park for P5.
- **Next: P2 (volatility signals)** — BB width and ATR will help size positions
  and detect when the band-touch signal is more reliable (compressed bands = pre-breakout,
  wide bands = high-risk entry).
